In [1]:
import sys 
sys.path.append('../../..')
from main_multitask_multimodal import LLM_MultitaskMultimodal

In [2]:
import json

In [3]:
model = LLM_MultitaskMultimodal.load_from_checkpoint('../../../models/multitask_multimodal_multilingual/version_11/epoch=3-step=776.ckpt'
                                                     ).cuda().eval()

INFO:root:Unused kwargs when getting microsoft/Multilingual-MiniLM-L12-H384: {'distance_func': 'cosine', 'loss_type': 'cross-entropy', 'margin': None, 'hidden_states_type': 'encoder-first', 'add_simcse': False, 'manual_loss_type': 'manual_mse', 'auto_task_weight': False, 'multitask_specs_dict': {'seqclf_singlemodal_wishquery2tax': {'specs': {'clf_head': 'clf_head', 'clf_weight_type': None, 'label_map_file': '/workspaces/multitask-llm-rnd/modelling/datasets/taxonomy/wish_v1.2.1_newtax_allpaths_withunknown.txt', 'label_type': 'multilabel_taxonomy'}}}, 'head_dict': {'clf_head': {'purpose': 'seqclf', 'type': 'linear', 'in_features': 384, 'out_features': 6038}}}


# qualitative check

In [4]:
inputs = model.tokenizer("Classify query: dildo", return_tensors='pt').to('cuda')

In [5]:
probs = model.clf_head(model.get_hidden_states(**inputs)).sigmoid()

In [6]:
top_probs, top_preds = probs.topk(10)

In [7]:
top_preds

tensor([[1130, 1121, 1146, 1149, 1143, 1140, 1116, 1126, 1125, 1145]],
       device='cuda:0')

In [8]:
label_map_file = model.hparams.multitask_specs_dict['seqclf_singlemodal_wishquery2tax']['specs']['label_map_file']

In [20]:
model.hparams.multitask_specs_dict['seqclf_singlemodal_wishquery2tax']['specs']['label_map_file']

'/workspaces/multitask-llm-rnd/modelling/datasets/taxonomy/wish_v1.2.1_newtax_allpaths_withunknown.txt'

In [9]:
label_map = {}
with open(label_map_file, 'r') as f:
    for l in f:
        l = l.replace('\n', '').strip()
        if len(l):
            label_map[l] = len(label_map)

In [21]:
label_map['unknown']

6037

In [10]:
label_map_rev = {label_map[i]: i for i in label_map}

In [11]:
[label_map_rev[i.item()] for i in top_preds[0]]

['beauty & health > sexual wellness > sex toys > dildos',
 'beauty & health > sexual wellness > sex toys',
 'beauty & health > sexual wellness > sex toys > vibrators',
 'beauty & health > sexual wellness > sm products > bondage gear',
 'beauty & health > sexual wellness > sex toys > sex sucking toys',
 'beauty & health > sexual wellness > sex toys > pumps & enlargers',
 'beauty & health > sexual wellness',
 'beauty & health > sexual wellness > sex toys > anal sex toys',
 'beauty & health > sexual wellness > sex toys > anal plug',
 'beauty & health > sexual wellness > sex toys > vagina balls']

In [12]:
top_probs

tensor([[0.9914, 0.0414, 0.0127, 0.0028, 0.0027, 0.0027, 0.0022, 0.0022, 0.0021,
         0.0020]], device='cuda:0', grad_fn=<TopkBackward0>)

In [13]:
def infer_model(query, topk = 3):
    assert topk <= len(label_map_rev)
    inputs = model.tokenizer(f"Classify query: {query}", return_tensors='pt').to('cuda')
    probs = model.clf_head(model.get_hidden_states(**inputs)).sigmoid()
    top_probs, top_preds = probs.topk(topk)
    return [(label_map_rev[top_preds[0][ind].item()], top_probs[0][ind].item()) for ind in range(topk)]

In [14]:
infer_model('apple', 3)

[('cellphones & telecommunications > iphones', 0.8689472079277039),
 ('cellphones & telecommunications > mobile phone accessories',
  0.22444188594818115),
 ('consumer electronics > smart electronics > wearable devices > smart watches',
  0.15000593662261963)]

In [15]:
infer_model('iphone', 3)

[('cellphones & telecommunications > iphones', 0.9730964303016663),
 ('cellphones & telecommunications > mobile phone accessories',
  0.16338349878787994),
 ('cellphones & telecommunications > cellphones', 0.04844042286276817)]

In [16]:
badsearch = [
    "chopsticks",
    "keyboard for mac",
    "Logitech K380",
    "K380",
    "keyboard with usb dongle",
    "rc cars 1/10 scale brushless"
]

In [17]:
for q in badsearch:
    print(f" > {q}")
    print(infer_model(q.lower().strip(), 3))

 > chopsticks
[('home & garden > kitchen,dining & bar > flatware > chopsticks', 0.9727470874786377), ('home & garden > kitchen,dining & bar > disposable kitchenware > disposable chopsticks', 0.09758913516998291), ('home & garden > kitchen,dining & bar > kitchen tools & gadgets > sushi tools', 0.02138880454003811)]
 > keyboard for mac
[('computer & office > mouse & keyboards > keyboards', 0.733725368976593), ('computer & office > mouse & keyboards > keyboard mouse combos', 0.0785374566912651), ('computer & office > mouse & keyboards > mice & keyboards accessories', 0.07391010969877243)]
 > Logitech K380
[('computer & office > mouse & keyboards > mice', 0.938803493976593), ('computer & office > mouse & keyboards > mice & keyboards accessories', 0.12057773023843765), ('computer & office > laptop accessories > laptop mice', 0.05699257180094719)]
 > K380
[('security & protection > video surveillance > surveillance cameras', 0.43163105845451355), ('security & protection > security alarm > se

In [18]:
infer_model('Logitech K380'), infer_model('Logitech K380'.lower())

([('sports > hunting > hunting gun accessories', 0.1675172597169876),
  ('tools > hand tools > knives', 0.061903852969408035),
  ('security & protection > self defense supplies', 0.0551854632794857)],
 [('computer & office > mouse & keyboards > mice', 0.938803493976593),
  ('computer & office > mouse & keyboards > mice & keyboards accessories',
   0.12057773023843765),
  ('computer & office > laptop accessories > laptop mice',
   0.05699257180094719)])

# convert

In [19]:
import pandas as pd 
import dvc.api
from torch import nn
import torch

In [22]:
df_tax = pd.read_json(dvc.api.get_url(
    'datasets/data/taxonomy/wish_newtax.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)

INFO:aiobotocore.credentials:Found credentials in environment variables.


In [23]:
orig_cat2id = {}
lower2orig = {}
for i in df_tax.to_dict('records'):
    if len(i['category_path']) > 0:
        orig_cat2id[i['category_path'].lower()] = int(i['id'])
        lower2orig[i['category_path'].lower()] = i['category_path']
orig_cat2id['unknown'] = -1
lower2orig['unknown'] = 'unknown'

In [24]:
modelid2origid = {}
for i in label_map_rev:
    modelid2origid[i] = (orig_cat2id[label_map_rev[i]], lower2orig[label_map_rev[i]])
assert max(modelid2origid) == len(modelid2origid) - 1 and min(modelid2origid) == 0

In [26]:
modelid2origid[len(modelid2origid)-1]

(-1, 'unknown')

In [27]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

In [28]:
model_hf = AutoModelForSequenceClassification.from_pretrained("microsoft/Multilingual-MiniLM-L12-H384", 
                                                              id2label=modelid2origid).cuda().eval()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/Multilingual-MiniLM-L12-H384 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
tokenizer_hf = AutoTokenizer.from_pretrained("microsoft/Multilingual-MiniLM-L12-H384", fast=True)

In [30]:
set(model.transformer.state_dict().keys()) == set(model_hf.bert.state_dict().keys())

True

In [31]:
model_hf.bert.load_state_dict(model.transformer.state_dict())

<All keys matched successfully>

In [32]:
set(model.clf_head[1].state_dict().keys()) == set(model_hf.classifier.state_dict().keys())

True

In [33]:
model_hf.classifier.load_state_dict(model.clf_head[1].state_dict())

<All keys matched successfully>

In [34]:
with torch.no_grad():
    model_hf.bert.pooler.dense.weight = nn.Parameter(torch.eye(384).cuda())
    model_hf.bert.pooler.activation = nn.Identity()
    model_hf.bert.pooler.dense.bias = nn.Parameter(model_hf.bert.pooler.dense.bias * 0)

In [35]:
classify = pipeline("text-classification", model=model_hf,
        tokenizer=tokenizer_hf, function_to_apply='sigmoid', device=0)

In [36]:
classify('Classify query: apple', top_k=3)

[{'label': (1316, 'Cellphones & Telecommunications > iPhones'),
  'score': 0.8689472079277039},
 {'label': (1255,
   'Cellphones & Telecommunications > Mobile Phone Accessories'),
  'score': 0.22444191575050354},
 {'label': (1752,
   'Consumer Electronics > Smart Electronics > Wearable Devices > Smart Watches'),
  'score': 0.15000595152378082}]

In [37]:
infer_model('apple', 3)

[('cellphones & telecommunications > iphones', 0.8689472079277039),
 ('cellphones & telecommunications > mobile phone accessories',
  0.22444188594818115),
 ('consumer electronics > smart electronics > wearable devices > smart watches',
  0.15000593662261963)]

In [38]:
def infer_model_hf(query, topk = 3):
    assert topk <= len(label_map_rev)
    inputs = tokenizer_hf(f"Classify query: {query}", return_tensors='pt').to('cuda')
    probs = model_hf(**inputs).logits.sigmoid()
    top_probs, top_preds = probs.topk(topk)
    return [(label_map_rev[top_preds[0][ind].item()], top_probs[0][ind].item()) for ind in range(topk)]

In [39]:
infer_model_hf('apple', 3)

[('cellphones & telecommunications > iphones', 0.8689472079277039),
 ('cellphones & telecommunications > mobile phone accessories',
  0.22444188594818115),
 ('consumer electronics > smart electronics > wearable devices > smart watches',
  0.15000593662261963)]

In [40]:
%%timeit
classify('Classify query: apple', top_k=3)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


17.8 ms ± 198 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [41]:
%%timeit
infer_model('apple', 3)

10.9 ms ± 26.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [42]:
%%timeit
infer_model_hf('apple', 3)

10.9 ms ± 51.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [43]:
infer_model_hf('apple', 3)

[('cellphones & telecommunications > iphones', 0.8689472079277039),
 ('cellphones & telecommunications > mobile phone accessories',
  0.22444188594818115),
 ('consumer electronics > smart electronics > wearable devices > smart watches',
  0.15000593662261963)]

In [44]:
for q in badsearch:
    b = json.dumps(infer_model_hf('apple', 3))
    c = json.dumps(infer_model('apple', 3))
    assert b==c

In [45]:
model_hf.save_pretrained('hf_ckpt')

In [46]:
tokenizer_hf.save_pretrained('hf_ckpt')

('hf_ckpt/tokenizer_config.json',
 'hf_ckpt/special_tokens_map.json',
 'hf_ckpt/sentencepiece.bpe.model',
 'hf_ckpt/added_tokens.json',
 'hf_ckpt/tokenizer.json')

In [47]:
model_hf_reload = AutoModelForSequenceClassification.from_pretrained('hf_ckpt').cuda().eval()

In [48]:
tokenizer_hf_reload = AutoTokenizer.from_pretrained('hf_ckpt')

In [49]:
def infer_model_hf_reload(query, topk = 3):
    assert topk <= len(label_map_rev)
    inputs = tokenizer_hf_reload(f"Classify query: {query}", return_tensors='pt').to('cuda')
    probs = model_hf_reload(**inputs).logits.sigmoid()
    top_probs, top_preds = probs.topk(topk)
    return [(label_map_rev[top_preds[0][ind].item()], top_probs[0][ind].item()) for ind in range(topk)]

In [50]:
infer_model_hf_reload('apple')

[('cellphones & telecommunications > iphones', 0.5747516751289368),
 ('consumer electronics > smart electronics > wearable devices > smart watches',
  0.2517867982387543),
 ('cellphones & telecommunications > mobile phone accessories',
  0.2467278242111206)]

In [51]:
with torch.no_grad():
    model_hf_reload.bert.pooler.activation = nn.Identity()

In [52]:
infer_model_hf_reload('apple')

[('cellphones & telecommunications > iphones', 0.8689472079277039),
 ('cellphones & telecommunications > mobile phone accessories',
  0.22444188594818115),
 ('consumer electronics > smart electronics > wearable devices > smart watches',
  0.15000593662261963)]

In [53]:
for q in badsearch:
    b = json.dumps(infer_model_hf('apple', 3))
    c = json.dumps(infer_model('apple', 3))
    d = json.dumps(infer_model_hf_reload('apple', 3))
    assert b==c==d

In [56]:
from hf_ckpt.load_model import load_pipeline

In [57]:
classify_reload = load_pipeline()

In [65]:
for q in badsearch:
    a = classify(f"Classify query: {q}")
    b = classify_reload(f"Classify query: {q}")
    c = infer_model(q)
    d = infer_model_hf(q)
    e = infer_model_hf_reload(q)
    assert json.dumps(a)==json.dumps(b)
    assert -1e-5 < c[0][1] - d[0][1] < 1e-5
    assert -1e-5 < d[0][1] - e[0][1] < 1e-5
    assert -1e-5 < e[0][1] - a[0]['score'] < 1e-5 
    assert -1e-5 < a[0]['score'] - b[0]['score'] < 1e-5 